In [1]:
import os
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        y = x
        x = self.fc2(x)
        return x, y

    def name(self):
        return "LeNet"


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(500, 128),
            nn.Tanh(),
            nn.Linear(128, 64))
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 500),
            nn.ReLU())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class DefenseNet(nn.Module):
    def __init__(self):
        super(DefenseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.ae1 = nn.Linear(500, 128)
        self.ae2 = nn.Linear(128, 64)
        self.ae3 = nn.Linear(64, 128)
        self.ae4 = nn.Linear(128, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = nn.Tanh(self.ae1(x))
        x = self.ae2(x)
        x = nn.Tanh(self.ae3(x))
        x = nn.ReLU(self.ae4(x))
        x = self.fc2(x)
        return x

In [3]:
model_le = torch.load('LeNet.pth')
model_ae = torch.load('AEC.pth')

/Users/blackielee/miniconda3/envs/name_of_my_env/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type LeNet. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
/Users/blackielee/miniconda3/envs/name_of_my_env/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type autoencoder. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [7]:
model_le.state_dict().keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [21]:
model_ae.state_dict().keys()

odict_keys(['encoder.0.weight', 'encoder.0.bias', 'encoder.2.weight', 'encoder.2.bias', 'decoder.0.weight', 'decoder.0.bias', 'decoder.2.weight', 'decoder.2.bias'])

In [8]:
dfnet = DefenseNet()

In [22]:
dfnet.state_dict().keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'ae1.weight', 'ae1.bias', 'ae2.weight', 'ae2.bias', 'ae3.weight', 'ae3.bias', 'ae4.weight', 'ae4.bias', 'fc2.weight', 'fc2.bias'])

In [24]:
dfnet.conv1.weight = torch.nn.Parameter(model_le.state_dict()['conv1.weight'])
dfnet.conv1.bias = torch.nn.Parameter(model_le.state_dict()['conv1.bias'])
dfnet.conv2.weight = torch.nn.Parameter(model_le.state_dict()['conv2.weight'])
dfnet.conv2.bias = torch.nn.Parameter(model_le.state_dict()['conv2.bias'])
dfnet.fc1.weight = torch.nn.Parameter(model_le.state_dict()['fc1.weight'])
dfnet.fc1.bias = torch.nn.Parameter(model_le.state_dict()['fc1.bias'])
dfnet.fc2.weight = torch.nn.Parameter(model_le.state_dict()['fc2.weight'])
dfnet.fc2.bias = torch.nn.Parameter(model_le.state_dict()['fc2.bias'])



In [25]:
dfnet.ae1.weight = torch.nn.Parameter(model_ae.state_dict()['encoder.0.weight'])
dfnet.ae1.bias = torch.nn.Parameter(model_ae.state_dict()['encoder.0.bias'])
dfnet.ae2.weight = torch.nn.Parameter(model_ae.state_dict()['encoder.2.weight'])
dfnet.ae2.bias = torch.nn.Parameter(model_ae.state_dict()['encoder.2.bias'])

dfnet.ae3.weight = torch.nn.Parameter(model_ae.state_dict()['decoder.0.weight'])
dfnet.ae3.bias = torch.nn.Parameter(model_ae.state_dict()['decoder.0.bias'])
dfnet.ae4.weight = torch.nn.Parameter(model_ae.state_dict()['decoder.2.weight'])
dfnet.ae4.bias = torch.nn.Parameter(model_ae.state_dict()['decoder.2.bias'])

In [26]:
torch.save(dfnet, 'DefenseNet.pth')

/Users/blackielee/miniconda3/envs/name_of_my_env/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type DefenseNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
